SMALLEST DOMAIN HEURISTIC

In [ ]:
def solve_hospital_assignment_with_heuristic(hosp_data, pat_data):
    nHosp = len(hosp_data)
    nPats = len(pat_data)
    max_beds = hosp_data['Camas'].tolist()
    hospital_coords = list(zip(hosp_data['Latitud'], hosp_data['Longitud']))
    patient_coords = list(zip(pat_data['Latitud'], pat_data['Longitud']))
    severities = pat_data['Severidad'].tolist()

    model = cp_model.CpModel()

    assignment = np.array([[model.NewBoolVar(f'x_{i}_{j}') for j in range(nPats)] for i in range(nHosp)])

    # Smallest domain heuristic
    all_vars = [assignment[i, j] for i in range(nHosp) for j in range(nPats)]
    model.AddDecisionStrategy(all_vars, cp_model.CHOOSE_FIRST, cp_model.SELECT_MIN_VALUE)

    for i in range(nHosp):
        model.Add(sum(assignment[i]) <= max_beds[i])

    for j in range(nPats):
        model.Add(sum(assignment[:, j]) == 1)

    total_distance = sum(
        sum(assignment[i][j] * np.linalg.norm(np.array(hospital_coords[i]) - np.array(patient_coords[j])) for j in range(nPats))
        for i in range(nHosp)
    )

    total_severity = sum(
        cp_model.LinearExpr.WeightedSum(assignment[i], [severities[j] for j in range(nPats)])
        for i in range(nHosp)
    )

    model.Minimize(total_distance + total_severity)

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL:
        assignments = {
            f'Hospital_{i}': [j for j in range(nPats) if solver.Value(assignment[i, j])]
            for i in range(nHosp)
        }
        return assignments
    else:
        return None

solution_with_heuristic = solve_hospital_assignment_with_heuristic(df, patients)